In [4]:
from datasets import load_dataset

ds = load_dataset("SemEvalWorkshop/sem_eval_2010_task_8")

README.md:   0%|          | 0.00/8.23k [00:00<?, ?B/s]

C:\Users\15278\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\15278\.cache\huggingface\hub\datasets--SemEvalWorkshop--sem_eval_2010_task_8. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/673k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/231k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2717 [00:00<?, ? examples/s]

In [7]:
ds

DatasetDict({
    train: Dataset({
        features: ['sentence', 'relation'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['sentence', 'relation'],
        num_rows: 2717
    })
})

In [13]:
print(ds["train"][:5])

{'sentence': ['The system as described above has its greatest application in an arrayed <e1>configuration</e1> of antenna <e2>elements</e2>.', 'The <e1>child</e1> was carefully wrapped and bound into the <e2>cradle</e2> by means of a cord.', 'The <e1>author</e1> of a keygen uses a <e2>disassembler</e2> to look at the raw assembly code.', 'A misty <e1>ridge</e1> uprises from the <e2>surge</e2>.', 'The <e1>student</e1> <e2>association</e2> is the voice of the undergraduate student population of the State University of New York at Buffalo.'], 'relation': [3, 18, 11, 18, 12]}


---

In [1]:
import re
import numpy as np
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import nltk

nltk.download("punkt")  # Ensure the nltk tokenizer is available


def extract_entities(sentence):
    """Extract entities within <e1> and <e2> tags and tokenize the sentence."""
    e1 = re.findall(r'<e1>(.*?)</e1>', sentence)[0]
    e2 = re.findall(r'<e2>(.*?)</e2>', sentence)[0]

    # Replace entity markers to ensure proper tokenization
    sentence = sentence.replace(f'<e1>{e1}</e1>', f' <e1> {e1} </e1> ')
    sentence = sentence.replace(f'<e2>{e2}</e2>', f' <e2> {e2} </e2> ')

    # Tokenize the sentence
    tokens = word_tokenize(sentence)

    return " ".join(tokens)  # Return the processed string


def preprocess_dataset(dataset):
    """Preprocess the dataset by extracting text and labels."""
    sentences = []
    labels = []
    
    for data in dataset:
        sentence = extract_entities(data["sentence"])  # Process the sentence
        relation = data["relation"]  # Extract the relation label
        sentences.append(sentence)
        labels.append(relation)

    return sentences, labels


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\15278\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
if __name__ == "__main__":
    # Load the SemEval-2010 Task 8 dataset
    dataset = load_dataset("SemEvalWorkshop/sem_eval_2010_task_8")

    # Preprocess the dataset
    train_sentences, train_labels = preprocess_dataset(dataset["train"])
    test_sentences, test_labels = preprocess_dataset(dataset["test"])

    vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=4000)  
    X_train = vectorizer.fit_transform(train_sentences)
    X_test = vectorizer.transform(test_sentences)

    svm_model = SVC(kernel="linear", C=5.0, class_weight="balanced", probability=True)
    svm_model.fit(X_train, train_labels)

    # predictions
    predictions = svm_model.predict(X_test)
    
    accuracy = accuracy_score(test_labels, predictions)
    print(f"Test set accuracy: {accuracy:.4f}\n")
    print("Classification report:")
    print(classification_report(test_labels, predictions, zero_division=0))


Test set accuracy: 0.6467

Classification report:
              precision    recall  f1-score   support

           0       0.84      0.76      0.80       134
           1       0.82      0.87      0.84       194
           2       0.60      0.75      0.67       162
           3       0.51      0.54      0.52       150
           4       0.76      0.86      0.81       153
           5       0.83      0.74      0.78        39
           6       0.78      0.86      0.82       291
           7       0.00      0.00      0.00         1
           8       0.72      0.81      0.76       211
           9       0.69      0.62      0.65        47
          10       0.43      0.45      0.44        22
          11       0.58      0.54      0.56       134
          12       0.69      0.34      0.46        32
          13       0.70      0.91      0.79       201
          14       0.65      0.71      0.68       210
          15       0.56      0.43      0.49        51
          16       0.61      0.

In [3]:
from scipy import linalg
triu = linalg.triu


pip install scipy==1.12
#https://stackoverflow.com/questions/78555551/importerror-cannot-import-name-triu-from-scipy-linalg

In [1]:
import re
import numpy as np
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import nltk
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import FunctionTransformer
from nltk.tag import hmm
from nltk.corpus import conll2000

nltk.download("punkt")  # Ensure the NLTK tokenizer is available
nltk.download("conll2000")  # Download the conll2000 dataset for HMM training


# HMM model class, used as a preprocessing step for SVM input features
class HMMFeatureExtractor(BaseEstimator, ClassifierMixin):
    def __init__(self):
        # Train a simple HMM model using NLTK's provided training data
        self.hmm_tagger = self.train_hmm()

    def train_hmm(self):
        # Train an HMM model using the conll2000 corpus
        train_data = conll2000.tagged_sents()
        trainer = hmm.HiddenMarkovModelTrainer()
        return trainer.train(train_data)  # Train with a limited number of iterations

    def transform(self, X):
        """Convert each sentence into a sequence of POS tags"""
        transformed_data = []
        for sentence in X:
            tokens = word_tokenize(sentence)
            tagged = self.hmm_tagger.tag(tokens)  # Get POS tags
            transformed_data.append([tag for _, tag in tagged])
        return transformed_data

    def fit(self, X, y=None):
        return self

    def predict(self, X):
        # Prediction function can be empty (we only care about feature extraction here)
        return np.zeros(len(X))


def extract_entities(sentence):
    """Extract entities from <e1> and <e2> tags and tokenize the sentence"""
    e1 = re.findall(r'<e1>(.*?)</e1>', sentence)[0]
    e2 = re.findall(r'<e2>(.*?)</e2>', sentence)[0]

    # Replace entity markers to make them recognizable
    sentence = sentence.replace(f'<e1>{e1}</e1>', f' <e1> {e1} </e1> ')
    sentence = sentence.replace(f'<e2>{e2}</e2>', f' <e2> {e2} </e2> ')

    # Tokenization
    tokens = word_tokenize(sentence)

    return " ".join(tokens)  # Return the processed string


def preprocess_dataset(dataset):
    """Preprocess the dataset by extracting text and labels"""
    sentences = []
    labels = []
    
    for data in dataset:
        sentence = extract_entities(data["sentence"])  # Process the sentence
        relation = data["relation"]  # Relation label
        sentences.append(sentence)
        labels.append(relation)

    return sentences, labels


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\15278\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\15278\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!


In [ ]:
if __name__ == "__main__":
    # Load the SemEval-2010 Task 8 dataset
    dataset = load_dataset("SemEvalWorkshop/sem_eval_2010_task_8")

    # Preprocess the dataset
    train_sentences, train_labels = preprocess_dataset(dataset["train"])
    test_sentences, test_labels = preprocess_dataset(dataset["test"])

    # TF-IDF vectorization
    vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=7000)
    X_train_tfidf = vectorizer.fit_transform(train_sentences)
    X_test_tfidf = vectorizer.transform(test_sentences)

    # Extract POS features using HMM
    hmm_extractor = HMMFeatureExtractor()
    X_train_hmm = hmm_extractor.transform(train_sentences)
    X_test_hmm = hmm_extractor.transform(test_sentences)

    # Encode POS tags using CountVectorizer
    pos_vectorizer = CountVectorizer(tokenizer=lambda x: x, lowercase=False)
    X_train_hmm_encoded = pos_vectorizer.fit_transform(X_train_hmm)
    X_test_hmm_encoded = pos_vectorizer.transform(X_test_hmm)

    # Combine TF-IDF features and HMM-based POS features
    X_train_combined = np.hstack([X_train_tfidf.toarray(), X_train_hmm_encoded.toarray()])
    X_test_combined = np.hstack([X_test_tfidf.toarray(), X_test_hmm_encoded.toarray()])

    # Train an SVM model
    svm_model = SVC(kernel="linear", C=5.0, class_weight="balanced", probability=True)
    svm_model.fit(X_train_combined, train_labels)

    # Make predictions
    predictions = svm_model.predict(X_test_combined)

    # Evaluate the model
    accuracy = accuracy_score(test_labels, predictions)
    print(f"Test Accuracy: {accuracy:.4f}\n")
    print("Classification Report:")
    print(classification_report(test_labels, predictions, zero_division=0))


HMMFeatureExtractor 生成的是 词性序列，然后用 CountVectorizer 进行编码。

CountVectorizer 的输出是固定维度的 词频矩阵（Bag-of-Words），不需要手动 padding。
SVM 直接处理这些固定维度的特征矩阵，不会因为序列长度不同而报错。

In [ ]:
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.tag import hmm
from nltk.corpus import conll2000

# Download necessary NLTK resources
nltk.download("punkt")  # Ensure word tokenizer is available
nltk.download("conll2000")  # Download conll2000 dataset for training HMM
nltk.download('punkt_tab')  # Download missing punkt_tab resource

# HMM model class for feature extraction before feeding data to the classifier
class HMMFeatureExtractor:
    def __init__(self):
        self.hmm_tagger = self.train_hmm()

    def train_hmm(self):
        train_data = conll2000.tagged_sents()
        trainer = hmm.HiddenMarkovModelTrainer()
        return trainer.train(train_data)  # Train an HMM tagger

    def transform(self, X):
        transformed_data = []
        for sentence in X:
            tokens = word_tokenize(sentence)
            tagged = self.hmm_tagger.tag(tokens)  # Get POS tags
            transformed_data.append([tag for _, tag in tagged])
        return transformed_data

    def fit(self, X, y=None):
        return self

    def predict(self, X):
        return np.zeros(len(X))


# Function to extract entities and tokenize sentences
def extract_entities(sentence):
    e1 = re.findall(r'<e1>(.*?)</e1>', sentence)[0]
    e2 = re.findall(r'<e2>(.*?)</e2>', sentence)[0]

    sentence = sentence.replace(f'<e1>{e1}</e1>', f' <e1> {e1} </e1> ')
    sentence = sentence.replace(f'<e2>{e2}</e2>', f' <e2> {e2} </e2> ')

    tokens = word_tokenize(sentence)

    return " ".join(tokens)


# Function to preprocess the dataset
def preprocess_dataset(dataset):
    sentences = []
    labels = []
    
    for data in dataset:
        sentence = extract_entities(data["sentence"])
        relation = data["relation"]
        sentences.append(sentence)
        labels.append(relation)

    return sentences, labels


# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


if __name__ == "__main__":
    # Load the SemEval-2010 Task 8 dataset
    dataset = load_dataset("SemEvalWorkshop/sem_eval_2010_task_8")

    # Preprocess the dataset
    train_sentences, train_labels = preprocess_dataset(dataset["train"])
    test_sentences, test_labels = preprocess_dataset(dataset["test"])

    # TF-IDF vectorization
    vectorizer = TfidfVectorizer(ngram_range=(1,3), max_features=20000)
    X_train_tfidf = vectorizer.fit_transform(train_sentences)
    X_test_tfidf = vectorizer.transform(test_sentences)

    # Extract POS features using HMM
    hmm_extractor = HMMFeatureExtractor()
    X_train_hmm = hmm_extractor.transform(train_sentences)
    X_test_hmm = hmm_extractor.transform(test_sentences)

    # Encode POS tags using CountVectorizer
    pos_vectorizer = CountVectorizer(tokenizer=lambda x: x, lowercase=False)
    X_train_hmm_encoded = pos_vectorizer.fit_transform(X_train_hmm)
    X_test_hmm_encoded = pos_vectorizer.transform(X_test_hmm)

    # Combine TF-IDF features and HMM-based POS features
    X_train_combined = np.hstack([X_train_tfidf.toarray(), X_train_hmm_encoded.toarray()])
    X_test_combined = np.hstack([X_test_tfidf.toarray(), X_test_hmm_encoded.toarray()])

    # Convert data to PyTorch tensors and transfer to GPU
    X_train_tensor = torch.tensor(X_train_combined, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test_combined, dtype=torch.float32).to(device)

    # Convert labels to numerical format
    label_mapping = {label: idx for idx, label in enumerate(set(train_labels))}
    y_train = torch.tensor([label_mapping[label] for label in train_labels], dtype=torch.long).to(device)
    y_test = torch.tensor([label_mapping[label] for label in test_labels], dtype=torch.long).to(device)

    # Create the model
    model = SimpleNN(input_dim=X_train_tensor.shape[1], output_dim=len(label_mapping)).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    num_epochs = 3000
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

    # Make predictions
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)

    # Evaluate the model
    accuracy = accuracy_score(y_test.cpu(), predicted.cpu())
    print(f"Test Accuracy: {accuracy:.4f}\n")

    # Print classification report
    print("Classification Report:")
    print(classification_report(y_test.cpu(), predicted.cpu(), target_names=[str(key) for key in label_mapping.keys()], zero_division=0))  # Convert keys to strings
